# Module 8: 6 - Anatomy of an AI Agent - Short-Term Memory
----------------------------------------------------------------------------
In this lesson, we will enhance our AI Agent by implementing a short-term memory feature. This will allow the agent to recall previous interactions within the conversation, providing more contextually relevant responses. We will define a class to manage conversation memory and integrate this with the existing agent structure, which includes the LLM Client using OpenAI models.

## Objectives
* Understand the concept of short-term memory in an AI agent.
* Define and implement a class to manage conversation memory.
* Integrate the memory management with the existing AI Agent.
* Test the AI Agent's ability to recall previous interactions and respond contextually.

## What this session covers:
* Installing necessary libraries for OpenAI integration.
* Defining the current agent structure including the LLM Client.
* Implementing short-term memory management for conversation context.
* Integrating the short-term memory with the AI Agent.
* Initializing and testing the enhanced AI Agent with example interactions.

## Install Libraries

In [2]:
#! pip install openai

## Define Current Agent Structure

### LLM Client

In [1]:
from typing import Dict, Any, List
import openai


class OpenAIChatCompletion:
    """
    Interacts with OpenAI's API for chat completions.
    """

    def __init__(self, model: str, api_key: str = None, base_url: str = None):
        """
        Initialize with model, API key, and base URL.
        """
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def generate(self, messages: List[Dict], **kwargs) -> Dict[str, Any]:
        """
        Generate a response from input messages.
        """
        params = {"messages": messages, "model": self.model, **kwargs}
        response = self.client.chat.completions.create(**params)
        return response.choices[0].message

### Agent Base

In [2]:
class Agent:
    """
    Integrates LLM client, tools, and memory.
    """

    def __init__(self, llm_client, tools=None, memory=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = memory

    def run(self, messages: List[Dict[str, str]]):
        """
        Generates a response from the LLM client.
        """
        # Generate response using the LLM client
        response = self.llm_client.generate(messages)
        return response

## Implementing Short-Term Memory

After setting up the LLM client, let's enable the agent to recall previous interactions. Short-term memory handles recent conversations within the LLM's context window, while long-term memory uses databases like vector stores or graphs for retrieving extensive chat history, ensuring relevant context.

In [3]:
from typing import List, Dict


class ChatMessageMemory:
    """Manages conversation context."""

    def __init__(self):
        self.messages = []

    def add_message(self, message: Dict):
        """Add a message to memory."""
        self.messages.append(message)

    def add_messages(self, messages: List[Dict]):
        """Add multiple messages to memory."""
        for message in messages:
            self.add_message(message)

    def get_messages(self) -> List[Dict]:
        """Retrieve all messages."""
        return self.messages.copy()

    def reset_memory(self):
        """Clear all messages."""
        self.messages = []

### Integrating Short-Term Memory with AI Agent

In [4]:
class Agent:
    """Integrates LLM client, tools, and memory."""

    def __init__(self, llm_client, system_message: Dict[str, str], tools=None):
        self.llm_client = llm_client
        self.tools = tools
        self.memory = ChatMessageMemory()
        self.system_message = system_message

    def run(self, user_message: Dict[str, str]):
        """Generate a response using LLM client and store context."""
        self.memory.add_message(user_message)
        chat_history = [self.system_message] + self.memory.get_messages()
        response = self.llm_client.generate(chat_history)
        self.memory.add_message(response)
        return response

### Testing Current Agent

Initialize Client

In [7]:
# API from environment variable
import os


api_key = os.environ.get("OPENAI_API_KEY")


client = OpenAIChatCompletion(

    base_url="https://api.openai.com/v1", model="gpt-4", api_key=api_key

)

Define System messages

In [8]:
# Define the system message
system_message = {"role": "system", "content": "You are a security assistant."}

Initialize New AI Agent

In [9]:
# Initialize the Agent with the LLM client and system message
agent = Agent(llm_client=client, system_message=system_message)

Send a User Message

In [10]:
# Define a user message
user_message = {"role": "user", "content": "Hey! This is Roberto!"}

# Generate a response using the agent
response = agent.run(user_message)

In [11]:
response.to_dict()

{'content': 'Hello Roberto! How can I assist you today?',
 'refusal': None,
 'role': 'assistant'}

Testing Short-Term Memory

In [12]:
# Add a follow-up message to test memory integration
follow_up_message = {"role": "user", "content": "What was my name?"}

# Run the agent again with the new message
response = agent.run(follow_up_message)
print(response)

ChatCompletionMessage(content='Your name is Roberto.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)


In [13]:
response.to_dict()

{'content': 'Your name is Roberto.', 'refusal': None, 'role': 'assistant'}